In [3]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm_notebook

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import KernelPCA
from sklearn.mixture import GaussianMixture as GMM
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import lightgbm as lgbm

In [12]:
class hist_model(object):
    
    def __init__(self, bins=50):
        self.bins = bins
        
    def fit(self, X):
        
        bin_hight, bin_edge = [], []
        
        for var in X.T:
            bh, bedge = np.histogram(var, bins=self.bins)
            bin_hight.append(bh)
            bin_edge.append(bedge)
        
        self.bin_hight = np.array(bin_hight)
        self.bin_edge = np.array(bin_edge)

    def predict(self, X):
        
        scores = []
        for obs in X:
            obs_score = []
            for i, var in enumerate(obs):
                bin_num = (var > self.bin_edge[i]).argmin()-1
                obs_score.append(self.bin_hight[i, bin_num])
            
            scores.append(np.mean(obs_score))
        
        return np.array(scores)

In [13]:
def run_model(clf_list, train, test, random_state, gmm_init_params='kmeans'):
    
    MODEL_COUNT = len(clf_list)
    
    oof_train = np.zeros((len(train), MODEL_COUNT))
    oof_test = np.zeros((len(test), MODEL_COUNT))
    train_columns = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]
    
    for magic in tqdm_notebook(range(512)):
        x_train = train[train['wheezy-copper-turtle-magic'] == magic]
        x_test = test[test['wheezy-copper-turtle-magic'] == magic]
        print("Magic: ", magic, x_train.shape, x_test.shape)
        
        train_idx_origin = x_train.index
        test_idx_origin = x_test.index
        
        train_std = x_train[train_columns].std()
        cols = list(train_std.index.values[np.where(train_std >2)])
        
        x_train = x_train.reset_index(drop=True)
        y_train = x_train.target
        
        x_train = x_train[cols].values
        x_test = x_test[cols].values
        
        all_data = np.vstack([x_train, x_test])
        
        all_data = KernelPCA(n_components=len(cols), kernel='cosine', random_state=random_state).fit_transform(all_data)
        
        gmm = GMM(n_components=5, random_state=random_state, max_iter=1000, init_params=gmm_init_params).fit(all_data)
        gmm_pred = gmm.predict_proba(all_data)
        gmm_score = gmm.score_samples(all_data).reshape(-1, 1)
        gmm_label = gmm.predict(all_data)
        
        hist = hist_model()
        hist.fit(all_data)
        hist_pred = hist.predict(all_data).reshape(-1, 1)
        
        all_data = np.hstack([all_data, gmm_pred, gmm_pred, gmm_pred, gmm_pred, gmm_pred])

        all_data = np.hstack([all_data, hist_pred, gmm_score, gmm_score, gmm_score])
        
        all_data = StandardScaler().fit_transform(all_data)

        x_train = all_data[:x_train.shape[0]]
        x_test = all_data[x_train.shape[0]:]
        fold = StratifiedKFold(n_splits=5, random_state=random_state)
        for trn_idx, val_idx in fold.split(x_train, gmm_label[:x_train.shape[0]]):
            for model_index, clf in enumerate(clf_list):
                clf.fit(x_train[trn_idx], y_train[trn_idx])
                oof_train[train_idx_origin[val_idx], model_index] = clf.predict_proba(x_train[val_idx])[:,1]
                
                if x_test.shape[0] == 0:
                    continue
                oof_test[test_idx_origin, model_index] += clf.predict_proba(x_test)[:,1] / fold.n_splits
    
    for i, clf in enumerate(clf_list):
        print(clf)
        print(roc_auc_score(train['target'], oof_train[:, i]))
        print()
        
    oof_train_df = pd.DataFrame(oof_train)
    oof_test_df = pd.DataFrame(oof_test)
    
    return oof_train_df, oof_test_df

In [7]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
submission = pd.read_csv('input/sample_submission.csv')

In [8]:
svnu_params = {'probability':True, 'kernel':'poly','degree':4,'gamma':'auto','nu':0.4,'coef0':0.08, 'random_state':4}
svnu2_params = {'probability':True, 'kernel':'poly','degree':2,'gamma':'auto','nu':0.4,'coef0':0.08, 'random_state':4}
qda_params = {'reg_param':0.111}
svc_params = {'probability':True,'kernel':'poly','degree':4,'gamma':'auto', 'random_state':4}
neighbor_params = {'n_neighbors':16}
lr_params = {'solver':'liblinear','penalty':'l1','C':0.05,'random_state':42}

In [14]:
nusvc_model = svm.NuSVC(**svnu_params)
nusvc2_model = svm.NuSVC(**svnu2_params)
qda_model = QuadraticDiscriminantAnalysis(**qda_params)
svc_model = svm.SVC(**svc_params)
knn_model = neighbors.KNeighborsClassifier(**neighbor_params)
lr_model = linear_model.LogisticRegression(**lr_params)

model_list = [nusvc_model, nusvc2_model, qda_model, svc_model, knn_model, lr_model]
oof_train_kmeans_seed1, oof_test_kmeans_seed1 = run_model(model_list, train, test, 1)
oof_train_kmeans_seed2, oof_test_kmeans_seed2 = run_model(model_list, train, test, 2)
oof_train_random_seed1, oof_test_random_seed1 = run_model(model_list, train, test, 1, 'random')
oof_train_random_seed2, oof_test_random_seed2 = run_model(model_list, train, test, 2, 'random')

Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

Magic:  260 (531, 258) (1024, 257)
Magic:  261 (508, 258) (1024, 257)
Magic:  262 (491, 258) (1024, 257)
Magic:  263 (516, 258) (1024, 257)
Magic:  264 (501, 258) (1024, 257)
Magic:  265 (500, 258) (1024, 257)
Magic:  266 (514, 258) (1024, 257)
Magic:  267 (507, 258) (1024, 257)
Magic:  268 (539, 258) (1024, 257)
Magic:  269 (508, 258) (1024, 257)
Magic:  270 (524, 258) (1024, 257)
Magic:  271 (499, 258) (1024, 257)
Magic:  272 (515, 258) (1024, 257)
Magic:  273 (504, 258) (1024, 257)
Magic:  274 (512, 258) (1024, 257)
Magic:  275 (532, 258) (1024, 257)
Magic:  276 (515, 258) (1024, 257)
Magic:  277 (476, 258) (1024, 257)
Magic:  278 (508, 258) (1024, 257)
Magic:  279 (484, 258) (1024, 257)
Magic:  280 (503, 258) (1024, 257)
Magic:  281 (518, 258) (1024, 257)
Magic:  282 (515, 258) (1024, 257)
Magic:  283 (504, 258) (1024, 257)
Magic:  284 (493, 258) (1024, 257)
Magic:  285 (531, 258) (1024, 257)
Magic:  286 (510, 258) (1024, 257)
Magic:  287 (511, 258) (1024, 257)
Magic:  288 (509, 25

Magic:  505 (515, 258) (0, 257)
Magic:  506 (533, 258) (0, 257)
Magic:  507 (519, 258) (0, 257)
Magic:  508 (522, 258) (0, 257)
Magic:  509 (501, 258) (0, 257)
Magic:  510 (515, 258) (0, 257)
Magic:  511 (502, 258) (0, 257)

NuSVC(coef0=0.08, degree=4, gamma='auto', kernel='poly', nu=0.4,
      probability=True, random_state=4)
0.9334908795707016

NuSVC(coef0=0.08, degree=2, gamma='auto', kernel='poly', nu=0.4,
      probability=True, random_state=4)
0.9385543153101795

QuadraticDiscriminantAnalysis(reg_param=0.111)
0.9636611087483075

SVC(degree=4, gamma='auto', kernel='poly', probability=True, random_state=4)
0.9306129697339929

KNeighborsClassifier(n_neighbors=16)
0.9001387261309955

LogisticRegression(C=0.05, penalty='l1', random_state=42, solver='liblinear')
0.83957078663201



Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

Magic:  260 (531, 258) (1024, 257)
Magic:  261 (508, 258) (1024, 257)
Magic:  262 (491, 258) (1024, 257)
Magic:  263 (516, 258) (1024, 257)
Magic:  264 (501, 258) (1024, 257)
Magic:  265 (500, 258) (1024, 257)
Magic:  266 (514, 258) (1024, 257)
Magic:  267 (507, 258) (1024, 257)
Magic:  268 (539, 258) (1024, 257)
Magic:  269 (508, 258) (1024, 257)
Magic:  270 (524, 258) (1024, 257)
Magic:  271 (499, 258) (1024, 257)
Magic:  272 (515, 258) (1024, 257)
Magic:  273 (504, 258) (1024, 257)
Magic:  274 (512, 258) (1024, 257)
Magic:  275 (532, 258) (1024, 257)
Magic:  276 (515, 258) (1024, 257)
Magic:  277 (476, 258) (1024, 257)
Magic:  278 (508, 258) (1024, 257)
Magic:  279 (484, 258) (1024, 257)
Magic:  280 (503, 258) (1024, 257)
Magic:  281 (518, 258) (1024, 257)
Magic:  282 (515, 258) (1024, 257)
Magic:  283 (504, 258) (1024, 257)
Magic:  284 (493, 258) (1024, 257)
Magic:  285 (531, 258) (1024, 257)
Magic:  286 (510, 258) (1024, 257)
Magic:  287 (511, 258) (1024, 257)
Magic:  288 (509, 25

...

Magic:  260 (531, 258) (1024, 257)
Magic:  261 (508, 258) (1024, 257)
Magic:  262 (491, 258) (1024, 257)
Magic:  263 (516, 258) (1024, 257)
Magic:  264 (501, 258) (1024, 257)
Magic:  265 (500, 258) (1024, 257)
Magic:  266 (514, 258) (1024, 257)
Magic:  267 (507, 258) (1024, 257)
Magic:  268 (539, 258) (1024, 257)
Magic:  269 (508, 258) (1024, 257)
Magic:  270 (524, 258) (1024, 257)
Magic:  271 (499, 258) (1024, 257)
Magic:  272 (515, 258) (1024, 257)
Magic:  273 (504, 258) (1024, 257)
Magic:  274 (512, 258) (1024, 257)
Magic:  275 (532, 258) (1024, 257)
Magic:  276 (515, 258) (1024, 257)
Magic:  277 (476, 258) (1024, 257)
Magic:  278 (508, 258) (1024, 257)
Magic:  279 (484, 258) (1024, 257)
Magic:  280 (503, 258) (1024, 257)
Magic:  281 (518, 258) (1024, 257)
Magic:  282 (515, 258) (1024, 257)
Magic:  283 (504, 258) (1024, 257)
Magic:  284 (493, 258) (1024, 257)
Magic:  285 (531, 258) (1024, 257)
Magic:  286 (510, 258) (1024, 257)
Magic:  287 (511, 258) (1024, 257)
Magic:  288 (509, 25

Magic:  505 (515, 258) (0, 257)
Magic:  506 (533, 258) (0, 257)
Magic:  507 (519, 258) (0, 257)
Magic:  508 (522, 258) (0, 257)
Magic:  509 (501, 258) (0, 257)
Magic:  510 (515, 258) (0, 257)
Magic:  511 (502, 258) (0, 257)

NuSVC(coef0=0.08, degree=4, gamma='auto', kernel='poly', nu=0.4,
      probability=True, random_state=4)
0.920690174151823

NuSVC(coef0=0.08, degree=2, gamma='auto', kernel='poly', nu=0.4,
      probability=True, random_state=4)
0.9306193856165093

QuadraticDiscriminantAnalysis(reg_param=0.111)
0.9623798466662554

SVC(degree=4, gamma='auto', kernel='poly', probability=True, random_state=4)
0.9175617655583486

KNeighborsClassifier(n_neighbors=16)
0.8768770438274944

LogisticRegression(C=0.05, penalty='l1', random_state=42, solver='liblinear')
0.8434448471835065



Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

Magic:  260 (531, 258) (1024, 257)
Magic:  261 (508, 258) (1024, 257)
Magic:  262 (491, 258) (1024, 257)
Magic:  263 (516, 258) (1024, 257)
Magic:  264 (501, 258) (1024, 257)
Magic:  265 (500, 258) (1024, 257)
Magic:  266 (514, 258) (1024, 257)
Magic:  267 (507, 258) (1024, 257)
Magic:  268 (539, 258) (1024, 257)
Magic:  269 (508, 258) (1024, 257)
Magic:  270 (524, 258) (1024, 257)
Magic:  271 (499, 258) (1024, 257)
Magic:  272 (515, 258) (1024, 257)
Magic:  273 (504, 258) (1024, 257)
Magic:  274 (512, 258) (1024, 257)
Magic:  275 (532, 258) (1024, 257)
Magic:  276 (515, 258) (1024, 257)
Magic:  277 (476, 258) (1024, 257)
Magic:  278 (508, 258) (1024, 257)
Magic:  279 (484, 258) (1024, 257)
Magic:  280 (503, 258) (1024, 257)
Magic:  281 (518, 258) (1024, 257)
Magic:  282 (515, 258) (1024, 257)
Magic:  283 (504, 258) (1024, 257)
Magic:  284 (493, 258) (1024, 257)
Magic:  285 (531, 258) (1024, 257)
Magic:  286 (510, 258) (1024, 257)
Magic:  287 (511, 258) (1024, 257)
Magic:  288 (509, 25

Magic:  505 (515, 258) (0, 257)
Magic:  506 (533, 258) (0, 257)
Magic:  507 (519, 258) (0, 257)
Magic:  508 (522, 258) (0, 257)
Magic:  509 (501, 258) (0, 257)
Magic:  510 (515, 258) (0, 257)
Magic:  511 (502, 258) (0, 257)

NuSVC(coef0=0.08, degree=4, gamma='auto', kernel='poly', nu=0.4,
      probability=True, random_state=4)
0.9211957688491367

NuSVC(coef0=0.08, degree=2, gamma='auto', kernel='poly', nu=0.4,
      probability=True, random_state=4)
0.9307176400225206

QuadraticDiscriminantAnalysis(reg_param=0.111)
0.9624968238903351

SVC(degree=4, gamma='auto', kernel='poly', probability=True, random_state=4)
0.9178227001359233

KNeighborsClassifier(n_neighbors=16)
0.8770075513668874

LogisticRegression(C=0.05, penalty='l1', random_state=42, solver='liblinear')
0.8397344944018273



In [15]:
train_second = (oof_train_kmeans_seed1 + oof_train_kmeans_seed2 + oof_train_random_seed1 + oof_train_random_seed2) / 4
test_second = (oof_test_kmeans_seed1 + oof_test_kmeans_seed2 + oof_test_random_seed1 + oof_test_random_seed2) / 4
print('Ensemble', roc_auc_score(train['target'], train_second.mean(1)))

Ensemble 0.9658454194494933


In [16]:
lgbm_meta_param = {
        #'bagging_freq': 5,
        #'bagging_fraction': 0.8,
        'min_child_weight':6.790,
        "subsample_for_bin":50000,
        'bagging_seed': 0,
        'boost_from_average':'true',
        'boost': 'gbdt',
        'feature_fraction': 0.450,
        'bagging_fraction': 0.343,
        'learning_rate': 0.025,
        'max_depth': 10,
        'metric':'auc',
        'min_data_in_leaf': 78,
        'min_sum_hessian_in_leaf': 8, 
        'num_leaves': 18,
        'num_threads': 8,
        'tree_learner': 'serial',
        'objective': 'binary', 
        'verbosity': 1,
        'lambda_l1': 7.961,
        'lambda_l2': 7.781
        #'reg_lambda': 0.3,
    }

mlp16_params = {'activation':'relu','solver':'lbfgs','tol':1e-06, 'hidden_layer_sizes':(16, ), 'random_state':42}

In [19]:
SEED_NUMBER = 4
NFOLD = 5

y_train = train['target']
oof_lgbm_meta_train = np.zeros((len(train), SEED_NUMBER))
oof_lgbm_meta_test = np.zeros((len(test), SEED_NUMBER))
oof_mlp_meta_train = np.zeros((len(train), SEED_NUMBER))
oof_mlp_meta_test = np.zeros((len(test), SEED_NUMBER))

for seed in range(SEED_NUMBER):
    print('SEED Ensemble:', seed)
    mlp16_params['random_state'] = seed
    lgbm_meta_param['seed'] = seed
    folds = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=seed)
    
    for fold_idx, (train_idx, val_idx) in enumerate(folds.split(train_second, y_train), 1):
        print(f'{fold_idx} FOLD Start')
        train_x, train_y = train_second.iloc[train_idx], y_train.iloc[train_idx]
        val_x, val_y = train_second.iloc[val_idx], y_train.iloc[val_idx]
        
        mlp_meta_model = neural_network.MLPClassifier(**mlp16_params)
        mlp_meta_model.fit(train_x, train_y)
        
        oof_mlp_meta_train[val_idx, seed] = mlp_meta_model.predict_proba(val_x)[:,1]
        oof_mlp_meta_test[:, seed] += mlp_meta_model.predict_proba(test_second)[:, 1] / NFOLD
        print('MLP META SCORE: ', roc_auc_score(val_y, oof_mlp_meta_train[val_idx, seed]))
        
        dtrain = lgbm.Dataset(train_x, label=train_y, silent=True)
        dcross = lgbm.Dataset(val_x, label=val_y, silent=True)
        
        lgbm_meta_model = lgbm.train(lgbm_meta_param, train_set=dtrain, valid_sets=[dtrain, dcross], verbose_eval=False, early_stopping_rounds=100)
        
        oof_lgbm_meta_train[val_idx, seed] = lgbm_meta_model.predict(val_x)
        oof_lgbm_meta_test[:, seed] += lgbm_meta_model.predict(test_second) / NFOLD
        print('LGBM META SCORE: ', roc_auc_score(val_y, oof_lgbm_meta_train[val_idx, seed]))

SEED Ensemble: 0
1 FOLD Start
MLP META SCORE:  0.9680613694753886
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=6.79 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=6.79 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Info] Number of positive: 104905, number of negative: 104810
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001576 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1340
[LightGBM] [Info] Number of data points in the train set: 209715, number of used features: 6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=6.79 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500226 -> initscore=0.000906
[LightGBM] [Info] Sta

LGBM META SCORE:  0.967348790041549
4 FOLD Start
MLP META SCORE:  0.9695506934726446
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=6.79 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=6.79 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Info] Number of positive: 104904, number of negative: 104811
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013799 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1340
[LightGBM] [Info] Number of data points in the train set: 209715, number of used features: 6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=6.79 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500222 -> initscore=0.000887
[LightGBM] [Info] Start training from score 0.000887
[LightGBM] [W

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
LGBM META SCORE:  0.967263800940314
2 FOLD Start
MLP META SCORE:  0.9693127302912588
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=6.79 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_weight=6.79 will be ignored. Current value: min_sum_hessian_in_leaf=8
[LightGBM] [Info] Number of positive: 104905, number of negative: 104810
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001939 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1340
[LightGBM] [Info] Number of data points in the train set: 209715, number of used features: 6
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=8, min_child_w

In [ ]:
oof_lgbm_meta_train_df = pd.DataFrame(oof_lgbm_meta_train).mean(axis=1).to_frame().rename(columns={0:'lgbm'})
oof_lgbm_meta_test_df = pd.DataFrame(oof_lgbm_meta_test).mean(axis=1).to_frame().rename(columns={0:'lgbm'})
oof_mlp_meta_train_df = pd.DataFrame(oof_mlp_meta_train).mean(axis=1).to_frame().rename(columns={0:'mlp'})
oof_mlp_meta_test_df = pd.DataFrame(oof_mlp_meta_test).mean(axis=1).to_frame().rename(columns={0:'mlp'})

In [ ]:
oof_train_third = pd.concat([train_second, oof_lgbm_meta_train_df, oof_mlp_meta_train_df], axis=1)
oof_test_third = pd.concat([test_second, oof_lgbm_meta_test_df, oof_mlp_meta_test_df], axis=1)

print('Ensemble', roc_auc_score(train['target'], oof_train_third.mean(1)))

In [ ]:
submission['target'] = oof_test_third.mean(1)
submission.to_csv('submission.csv', index=False)